# Using this in account B (Prod env) to create the resource.

#### Move all the Json files created in DEV env to the current directory in your notebook instance.


In [1]:
#importing the required libraries
import json
import boto3
import time
client  = boto3.client('quicksight')
s3=boto3.resource('s3')
account_id = boto3.client('sts').get_caller_identity()['Account']

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


In [2]:
# Provide the User ARN avaialble in the quicksight. 

response = client.list_users(AwsAccountId=account_id,Namespace='default')
response

{'ResponseMetadata': {'RequestId': 'c5cacffc-be41-445d-a4cc-88f8fc9b748e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sun, 07 Aug 2022 11:20:09 GMT',
   'content-type': 'application/json',
   'content-length': '2265',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'c5cacffc-be41-445d-a4cc-88f8fc9b748e'},
  'RetryAttempts': 0},
 'Status': 200,
 'UserList': [{'Arn': 'arn:aws:quicksight:us-east-1:508495579255:user/default/Dilip',
   'UserName': 'Dilip',
   'Email': 'r.kumar2@wipro.com',
   'Role': 'ADMIN',
   'IdentityType': 'IAM',
   'Active': True,
   'PrincipalId': 'federated/iam/AIDAJ5VSDCKPTA22IOYCU'},
  {'Arn': 'arn:aws:quicksight:us-east-1:508495579255:user/default/ajay.vishwakarma3@wipro.com',
   'UserName': 'ajay.vishwakarma3@wipro.com',
   'Email': 'ajay.vishwakarma3@wipro.com',
   'Role': 'ADMIN',
   'IdentityType': 'IAM',
   'Active': True,
   'PrincipalId': 'federated/iam/AIDAXMZFSZB3TGH4LRLH3'},
  {'Arn': 'arn:aws:quicksight:us-east-1:508495579255:user/defaul

In [3]:
# Functions required to create the resources

def create_datasource(client, account_id,Data_source_Id,data_source_name,type_ds,DataSourceParameter,user_arn,SslProperty):
    response = client.create_data_source(
    AwsAccountId=account_id,
    DataSourceId=Data_source_Id,      #Ds_id, #we'll provide the ds id
    Name=data_source_name, #display name for the data source
    Type=type_ds,
    DataSourceParameters=DataSourceParameter,
    #     Credentials=Credential,
    Permissions=[
    {'Principal': user_arn,
        'Actions': ["quicksight:DescribeDataSource","quicksight:DescribeDataSourcePermissions","quicksight:PassDataSource","quicksight:UpdateDataSource","quicksight:DeleteDataSource","quicksight:UpdateDataSourcePermissions"]}],
    #     VpcConnectionProperties={
    #         'VpcConnectionArn': 'string'
    #     },
    SslProperties=SslProperty)
    return response


def create_dataset(client, account_id, dataset_id,dataset_name,user_arn,Physical_TableMap,Logical_TableMap,Import_Mode,DataSet_Usage_Configuration ):
    output = client.create_data_set(
    AwsAccountId=account_id,
    DataSetId=dataset_id,
    Name=dataset_name, #dataset_name,
        Permissions=[
        {'Principal': user_arn,
            'Actions': ["quicksight:DescribeDataSet","quicksight:DescribeDataSetPermissions","quicksight:PassDataSet","quicksight:DescribeIngestion","quicksight:ListIngestions","quicksight:UpdateDataSet","quicksight:DeleteDataSet","quicksight:CreateIngestion","quicksight:CancelIngestion","quicksight:UpdateDataSetPermissions"]}],
    PhysicalTableMap=Physical_TableMap,
    LogicalTableMap=Logical_TableMap,
    ImportMode=Import_Mode,    
    DataSetUsageConfiguration=DataSet_Usage_Configuration )
    return output



def create_template_from_template(client,account_id,Template_Id,template_name,template_arn,user_arn):
    response = client.create_template(
    AwsAccountId=account_id,
    TemplateId=Template_Id,
    Name=template_name,
    Permissions=[
            {'Principal': user_arn,
                'Actions': ['quicksight:UpdateTemplatePermissions', 'quicksight:DescribeTemplatePermissions', 'quicksight:UpdateTemplateAlias', 'quicksight:DeleteTemplateAlias', 'quicksight:DescribeTemplateAlias', 'quicksight:ListTemplateAliases', 'quicksight:ListTemplates', 'quicksight:CreateTemplateAlias', 'quicksight:DeleteTemplate', 'quicksight:UpdateTemplate', 'quicksight:ListTemplateVersions', 'quicksight:DescribeTemplate', 'quicksight:CreateTemplate']}],    
            SourceEntity={
            'SourceTemplate': {
                'Arn': template_arn
            }
        })
    return response

def creating_analysis(client, account_id, analysis_id, analysis_name,dataset_ref, template_arn):
    response = client.create_analysis(
    AwsAccountId=account_id,
    AnalysisId=analysis_id,
    Name=analysis_name,
    SourceEntity={
        'SourceTemplate': {
            'DataSetReferences': dataset_ref,
            "Arn":template_arn,
                }
            }
        )
    return response

In [4]:
# Read and loads the datasource file
describe_datasource = open('datasource.json')
describe_datasource = json.load(describe_datasource)

In [6]:
#input parameters required
user_arn= 'arn:aws:quicksight:us-east-1:<account_Id>:user/default/<email.id.com>'
analysis_id = "Provide a unique ID for your analysis" 
analysis_name = "Provide a unique name for your analysis"

In [5]:
# We are taking input values from json file which we'll use to create a new datasource
ds_Id = describe_datasource['DataSource']['DataSourceId']
ds_Name = describe_datasource['DataSource']['Name']
ds_parameters = describe_datasource['DataSource']['DataSourceParameters']
ssl = describe_datasource['DataSource']['SslProperties']
type_ds = describe_datasource['DataSource']['Type']

datasource_response = create_datasource(client, account_id,ds_Id,ds_Name,type_ds,ds_parameters,user_arn,ssl)
time.sleep(20)
datasource_arn = datasource_response['Arn'] # ARN of the resource we'll use while creating dataset

In [6]:
# Read and load the dataset json file
describe_dataset = open('dataset.json')
describe_dataset = json.load(describe_dataset)

In [11]:
# fetching values required to create datasource from our json files
Import_Mode = describe_dataset['DataSet']['ImportMode']
Logical_TableMap=describe_dataset['DataSet']['LogicalTableMap']
Physical_TableMap=describe_dataset['DataSet']['PhysicalTableMap']
DataSet_Usage_Configuration = describe_dataset['DataSet']['DataSetUsageConfiguration']
dataset_Id = describe_dataset['DataSet']['DataSetId']
dataset_Name = describe_dataset['DataSet']['Name']

# Replacing the datasource Arn in the physical table map
for k,values in Physical_TableMap.items():
    diff_key = k
    mapping= Physical_TableMap[diff_key]
    for l,values in mapping.items():
        Physical_TableMap[diff_key][l]['DataSourceArn'] = datasource_arn
        

{'666e9667-40a4-45b6-bc35-4dd0430459b6': {'RelationalTable': {'DataSourceArn': 'datasource_arn',
   'Catalog': 'AwsDataCatalog',
   'Schema': 'modelmonitor-ghgwauarm0pf',
   'Name': 'scoringtrend',
   'InputColumns': [{'Name': 'runtime', 'Type': 'STRING'},
    {'Name': 'id', 'Type': 'STRING'},
    {'Name': 'year', 'Type': 'STRING'},
    {'Name': 'manufacturer', 'Type': 'STRING'},
    {'Name': 'model', 'Type': 'STRING'},
    {'Name': 'condition', 'Type': 'STRING'},
    {'Name': 'cylinders', 'Type': 'STRING'},
    {'Name': 'fuel', 'Type': 'STRING'},
    {'Name': 'odometer', 'Type': 'STRING'},
    {'Name': 'title_status', 'Type': 'STRING'},
    {'Name': 'transmission', 'Type': 'STRING'},
    {'Name': 'drive', 'Type': 'STRING'},
    {'Name': 'type', 'Type': 'STRING'},
    {'Name': 'paint_color', 'Type': 'STRING'},
    {'Name': 'predprice', 'Type': 'STRING'},
    {'Name': 'baseprice', 'Type': 'STRING'},
    {'Name': 'baseodometer', 'Type': 'STRING'},
    {'Name': 'basecylinders', 'Type': 'S

In [6]:
dataset_response = create_dataset(client, account_id,dataset_Id,dataset_Name,user_arn, Physical_TableMap,Logical_TableMap,Import_Mode,DataSet_Usage_Configuration)
time.sleep(20)
dataset_arn = dataset_response['Arn']

{'DisableUseAsDirectQuerySource': False, 'DisableUseAsImportedSource': False}

In [8]:
# Creating the template from the shared template

describe_template = open('template.json')
describe_template = json.load(describe_template)

describe_template={"ResponseMetadata": {"RequestId": "bab1a7ef-8ca6-41b7-8953-8cccd93fa170", "HTTPStatusCode": 200, "HTTPHeaders": {"date": "Tue, 28 Jun 2022 05:12:14 GMT", "content-type": "application/json", "content-length": "2963", "connection": "keep-alive", "x-amzn-requestid": "bab1a7ef-8ca6-41b7-8953-8cccd93fa170"}, "RetryAttempts": 0}, "Status": 200, "Template": {"Arn": "arn:aws:quicksight:us-east-1:525102048888:template/MM1", "Name": "MMM1", "Version": {"CreatedTime": "2022-06-24 12:26:05.863000+00:00", "VersionNumber": 1, "Status": "CREATION_SUCCESSFUL", "DataSetConfigurations": [{"Placeholder": "p1", "DataSetSchema": {"ColumnSchemaList": [{"Name": "infertype", "DataType": "STRING"}, {"Name": "bslineodometer", "DataType": "DECIMAL"}, {"Name": "transmission", "DataType": "STRING"}, {"Name": "monitorjobname", "DataType": "STRING"}, {"Name": "bslineprice", "DataType": "DECIMAL"}, {"Name": "vmodelname", "DataType": "STRING"}, {"Name": "scoringfilename", "DataType": "STRING"}, {"Name": "odometer", "DataType": "DECIMAL"}, {"Name": "monitorbatchid", "DataType": "INTEGER"}, {"Name": "predprice", "DataType": "DECIMAL"}, {"Name": "cylinders", "DataType": "STRING"}, {"Name": "batchid", "DataType": "INTEGER"}, {"Name": "modelname", "DataType": "STRING"}, {"Name": "inferenceid", "DataType": "STRING"}, {"Name": "runtime", "DataType": "DATETIME"}]}, "ColumnGroupSchemaList": []}, {"Placeholder": "p2", "DataSetSchema": {"ColumnSchemaList": [{"Name": "infertype", "DataType": "STRING"}, {"Name": "start_time", "DataType": "STRING"}, {"Name": "description", "DataType": "STRING"}, {"Name": "modelname", "DataType": "STRING"}, {"Name": "monitorjobname", "DataType": "STRING"}, {"Name": "feature_name", "DataType": "STRING"}]}, "ColumnGroupSchemaList": []}], "SourceEntityArn": "arn:aws:quicksight:us-east-1:525102048888:analysis/011d8c63-980d-4a6a-be28-2b1756ea72f5", "ThemeArn": "arn:aws:quicksight::aws:theme/MIDNIGHT", "Sheets": [{"SheetId": "d95e166e-8153-4e18-a763-91ef1befec96", "Name": "priceTrendPred"}, {"SheetId": "d29626b7-3a1e-4948-8fee-04afcd5fa40b", "Name": "ScoringDetailAnalysis"}, {"SheetId": "d6dee9bd-eff6-45d9-950f-bb94f045ad58", "Name": "MonitorJobDetail"}, {"SheetId": "94e31f8e-1f71-4bac-8a58-12e84f2ce0ee", "Name": "BatchIdDetails"}]}, "TemplateId": "MM1", "LastUpdatedTime": "2022-06-24 12:26:05.858000+00:00", "CreatedTime": "2022-06-24 12:26:05.858000+00:00"}, "RequestId": "bab1a7ef-8ca6-41b7-8953-8cccd93fa170"}

dev_tmpl_arn= describe_template['Template']['Arn']
template_name = describe_template['Template']['Name']
template_id = describe_template['Template']['TemplateId']
dataset_configuration = describe_template['Template']['Version']['DataSetConfigurations']
placeholder = describe_template['Template']['Version']['DataSetConfigurations'][0]['Placeholder']


In [16]:
template_response create_template_from_template(client,account_id,template_id,template_name,dev_tmpl_arn, user_arn)
template_arn = template_response['Arn']
time.sleep(20)

In [17]:
#Creating analysis from the template
dataset_ref = [{'DataSetPlaceholder':placeholder,'DataSetArn':dataset_arn}]

creating_analysis(client, account_id, analysis_id, analysis_name,dataset_ref, template_arn)

time.sleep(20)